## Конвертация YOLO-датасета в COCO фомат

In [2]:
from globox import AnnotationSet, Annotation, BoundingBox
from pathlib import Path, PosixPath
import numpy as np
from tqdm import tqdm

In [3]:
labels_path = Path("/mnt/work_share/DWH/DataSets/Detection/11/0/yolo/labels/test/")
images_path = Path("/mnt/work_share/DWH/DataSets/Detection/11/0/yolo/images/test/")
labels = Path("/mnt/work_share/DWH/DataSets/Detection/11/0/obj.names").read_text().splitlines()

In [ ]:
def find_open_img(imgs_path:PosixPath, name):
    from PIL import Image
    impath = imgs_path / name
    for ext in [".jpg", ".JPG", ".jpeg", ".JPEG", ".png", ".PNG"]:
        impath_ = impath.with_suffix(ext)
        if impath_.exists():
            try:
                return Image.open(impath_), impath_
            except:
                print("error open", impath_)
                pass
    return None, impath

annset = AnnotationSet()
lab_names = sorted([p for p in labels_path.glob("*.txt")])
for i, lab_path in enumerate(tqdm(lab_names)):
    img, img_path = find_open_img(images_path, lab_path.stem)
    if img is None:
        print("missing", img_path)
    else:
        w, h = img.size
        ann = Annotation(img_path.name, (w, h))
        ann_txt = Path(lab_path).read_text()
        for line in ann_txt.splitlines():
            cls_id, *bbox_str = line.split()
            x0, y0, bw, bh = map(float, bbox_str)
            bbox = BoundingBox(label=labels[int(cls_id)], 
                            xmin=w*(x0 - bw/2),
                            ymin=h*(y0 - bh/2),
                            xmax=w*(x0 + bw/2),
                            ymax=h*(y0 + bh/2),
                            confidence=1)
            print(bbox.is_ground_truth)
            ann.add(bbox)
    annset.add(ann, override=True)

In [7]:
print(annset.show_stats())

            Database Stats             
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Label              ┃ Images ┃ Boxes ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ <empty_image>      │     58 │     0 │
│ bicycle            │    328 │   979 │
│ car                │   1144 │  4163 │
│ dead_stop          │    119 │   123 │
│ derail             │    134 │   205 │
│ hand_shifter_left  │    307 │   465 │
│ hand_shifter_right │    284 │   450 │
│ letter             │   1401 │  1947 │
│ maneuver           │     34 │    35 │
│ milestone          │    396 │   397 │
│ motorcycle         │    190 │   398 │
│ person             │   1296 │  5675 │
│ pillar             │   3263 │  6109 │
│ sign               │     54 │    63 │
│ sound_signal       │    241 │   318 │
│ tflight            │   2502 │  6651 │
│ train              │   2471 │  5300 │
│ worker             │    505 │   749 │
├────────────────────┼────────┼───────┤
│ Total              │   7186 │ 34027 │
└────────────────────┴────────┴───────┘

None


In [8]:
annset._id_to_imageid = {i:img for i, img in enumerate(annset.image_ids)}
annset._id_to_label = {i:lab for i, lab in enumerate(labels)}
annset.save_coco("annotation.json")

## Пример сохранения детекций модели в json

In [ ]:
from globox import AnnotationSet, Annotation, BoundingBox

images_path = "Path/To/Dir/With/Images" #  TODO
class_names = ["label1", "label2", "labelN"] #  TODO
annset = AnnotationSet()
for img_path in Path(images_path).glob("*.jpg"):
    detections= get_model_predictions(img_path) #  TODO
    w, h = get_image_size(img_path) #  TODO
    ann = Annotation(img_path.name, (w, h))
    for bbox, class_id, conf in detections:
        bbox = BoundingBox(label=class_names[class_id], 
                           xmin=bbox[0],
                           ymin=bbox[1],
                           xmax=bbox[2],
                           ymax=bbox[3],
                           confidence=conf)
        ann.add(bbox)
    annset.add(ann)

annset._id_to_label = {i:lab for i, lab in enumerate(class_names)}
annset.save_coco("annotation.json")

## Простое вычисление COCO-метрик с помощью globox

In [8]:
from globox import AnnotationSet, COCOEvaluator

annset_gt = AnnotationSet.from_coco("/mnt/work_share/DWH/DataSets/Detection/11/1/labels/ann_det11.1_test.json")
annset_pr = AnnotationSet.from_coco("/mnt/work_share/developers/rybin/models/ObjectDetection/yolov7-1280-cls17/test/annotation.json")

evaluator = COCOEvaluator(ground_truths=annset_gt, predictions=annset_pr)
evaluator.show_summary()

                                                  COCO Evaluation                                                  
┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃        ┃     AP ┃        ┃        ┃        ┃        ┃        ┃        ┃       ┃        ┃       ┃        ┃       ┃
┃ Label  ┃  50:95 ┃  AP 50 ┃  AP 75 ┃   AP S ┃   AP M ┃   AP L ┃   AR 1 ┃ AR 10 ┃ AR 100 ┃  AR S ┃   AR M ┃  AR L ┃
┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ bicyc… │ 47.03% │ 75.87% │ 49.77% │ 24.59% │ 40.37% │ 57.14% │ 20.46% │ 44.3… │ 47.03% │ 35.3… │ 49.66% │ 65.3… │
│ car    │ 68.18% │ 90.73% │ 76.92% │ 51.86% │ 64.95% │ 76.37% │ 21.20% │ 65.5… │ 68.18% │ 64.9… │ 71.97% │ 80.8… │
│ dead_… │ 59.79% │ 92.11% │ 70.61% │ 56.16% │ 80.11% │   nan% │ 58.01% │ 59.7… │ 59.79% │ 63.5… │ 82.67% │  nan% │
│ derail │ 71.55% │ 93.01% │ 73.63% │ 40.63% │ 76.59% │ 92.18% │ 54.17% │ 71.5… │ 71.55% │ 45.7… │ 82.75% │ 93.9… │
│ hand_… │ 74.94% │ 96.84% │ 86.28% │ 46.34% │ 72.68% │ 83.66% │ 51.87% │ 74.9… │ 74.94% │ 61.6… │ 76.33% │ 86.4… │
│ hand_… │ 75.87% │ 96.79% │ 89.99% │ 40.80% │ 69.88% │ 83.00% │ 50.46% │ 75.8… │ 75.87% │ 69.3… │ 76.11% │ 86.7… │
│ letter │ 54.52% │ 87.54% │ 58.85% │ 45.16% │ 71.57% │ 80.74% │ 41.82% │ 54.5… │ 54.52% │ 52.9… │ 75.62% │ 83.7… │
│ maneu… │ 67.61% │ 93.79% │ 70.98% │ 59.29% │ 77.34% │ 100.0… │ 64.98% │ 67.6… │ 67.61% │ 64.2… │ 82.00% │ 100.… │
│ miles… │ 70.05% │ 93.53% │ 83.29% │ 47.59% │ 73.16% │ 75.09% │ 69.77% │ 70.0… │ 70.05% │ 57.8… │ 77.91% │ 79.0… │
│ motor… │ 46.29% │ 77.86% │ 49.16% │ 21.57% │ 33.47% │ 54.68% │ 25.12% │ 44.9… │ 46.29% │ 30.7… │ 41.95% │ 66.0… │
│ person │ 56.16% │ 86.24% │ 61.26% │ 35.75% │ 52.65% │ 69.15% │ 16.51% │ 47.1… │ 56.16% │ 50.5… │ 61.70% │ 75.4… │
│ pillar │ 55.26% │ 91.39% │ 60.03% │ 49.34% │ 71.09% │ 77.81% │ 34.46% │ 55.1… │ 55.26% │ 56.3… │ 76.52% │ 81.8… │
│ sign   │ 53.15% │ 79.78% │ 58.27% │ 44.28% │ 71.96% │ 40.40% │ 50.18% │ 53.1… │ 53.15% │ 52.7… │ 75.00% │ 40.0… │
│ sound… │ 76.95% │ 95.39% │ 92.21% │ 69.71% │ 81.30% │ 93.17% │ 59.94% │ 76.9… │ 76.95% │ 75.0… │ 84.63% │ 94.5… │
│ tflig… │ 57.07% │ 89.22% │ 61.57% │ 46.46% │ 71.30% │ 79.14% │ 24.72% │ 56.2… │ 57.07% │ 54.6… │ 75.96% │ 82.9… │
│ train  │ 54.72% │ 82.58% │ 58.00% │ 44.52% │ 50.44% │ 56.55% │ 27.85% │ 54.4… │ 54.72% │ 62.7… │ 64.69% │ 71.4… │
│ worker │ 69.42% │ 92.14% │ 78.40% │ 47.30% │ 65.20% │ 77.74% │ 49.38% │ 69.1… │ 69.42% │ 59.8… │ 71.65% │ 81.8… │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼───────┼────────┼───────┼────────┼───────┤
│ Total  │ 62.27% │ 89.11% │ 69.37% │ 45.37% │ 66.12% │ 74.80% │ 45.13% │ 66.9… │ 68.48% │ 56.3… │ 72.18% │ 79.3… │
└────────┴────────┴────────┴────────┴────────┴────────┴────────┴────────┴───────┴────────┴───────┴────────┴───────┘